In [ ]:
#79
from tqdm import tqdm
import torch, time
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from torch.utils.data import TensorDataset, DataLoader
torch.manual_seed(0)#シード固定しとく

#学習/評価データ読み込み
x_train = np.loadtxt("./x_train.txt", delimiter=" ")
y_train = np.loadtxt("./y_train.txt")
x_valid = np.loadtxt("./x_valid.txt", delimiter=" ")
y_valid = np.loadtxt("./y_valid.txt")
#pytorchのtensor型にする
x_train = torch.tensor(x_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.int64)
x_valid = torch.tensor(x_valid, dtype=torch.float32)
y_valid = torch.tensor(y_valid, dtype=torch.int64)

#ネットワーク作成
net = torch.nn.Sequential(
    torch.nn.Linear(300, 100),
    torch.nn.ReLU(),
    torch.nn.Linear(100, 20),
    torch.nn.ReLU(),
    torch.nn.Linear(20, 4)
)
optimizer = torch.optim.SGD(net.parameters(), lr=0.01)
loss_fn = torch.nn.CrossEntropyLoss()
batchsize = [64]

#データをタイトルとラベルの組ごとにタプルとして扱う
dataset = TensorDataset(x_train, y_train)

times = []

for bs in batchsize:
    loader = DataLoader(dataset, batch_size=bs, shuffle=True)#ミニバッチを返してくれる
    train_losses = []
    valid_losses = []
    train_accs = []
    valid_accs = []

    for epoch in tqdm(range(10)):
        start = time.time()#時間計測スタート
        for xx, yy in loader:
            optimizer.zero_grad()#勾配を0で初期化
            y_pred = net(x_train)#確率を計算
            loss = loss_fn(y_pred, y_train)#損失を計算
            loss.backward()#勾配を計算
            optimizer.step()#最適化ステップを実行

        #損失の記録
        train_losses.append(loss.detach().numpy())
        valid_losses.append(loss_fn(net(x_valid), y_valid).detach().numpy())
        #カテゴリの予測
        y_max_train, y_pred_train = torch.max(net(x_train),dim=1)
        y_max_valid, y_pred_valid = torch.max(net(x_valid),dim=1)
        #正解率の記録
        train_acc = accuracy_score(y_pred_train, y_train)
        valid_acc = accuracy_score(y_pred_valid, y_valid)
        train_accs.append(train_acc)
        valid_accs.append(valid_acc)

        #学習時間計測
        times.append(time.time() - start)

        #モデルを保存
        #torch.save(net.state_dict(), f"checkpoints/model_epoch{epoch}.pt")

100%|██████████| 10/10 [01:36<00:00,  9.66s/it]
